In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for subdir in ['merged_outputs','graphs']:
    if not os.path.exists(subdir):
        os.makedirs(subdir)

In [ ]:
input_files = os.path.join('input_files')
out_files = os.path.join('merged_outputs')

files_list = os.listdir(input_files)
print(files_list)

for i in range(0,len(files_list)):
    if files_list[i].endswith(".interfacea"):
        with open(input_files +"/"+ files_list[i], "r") as f:
            lines = f.readlines()
        
        with open(out_files +"/"+ "hydrophobics","a") as d:
            for j in range(1,len(lines)):
                if lines[j].split()[1] == 'hydrophobic':
                    for  k in range(1,len(lines[j].split())-1):
                        d.write(lines[j].split()[k] + "\t")
                    d.write(lines[j].split()[-1] +"\n")
                    
        with open(out_files +"/" + "hbonds","a") as d:
            for j in range(1,len(lines)):
                if lines[j].split()[1] == 'hbond':
                    for  k in range(1,len(lines[j].split())-1):
                        d.write(lines[j].split()[k] + "\t")
                    d.write(lines[j].split()[-1] +"\n")
                    
        with open(out_files +"/" + "ionics","a") as d:
            for j in range(1,len(lines)):
                if lines[j].split()[1] == 'ionic':
                    for  k in range(1,len(lines[j].split())-1):
                        d.write(lines[j].split()[k] + "\t")
                    d.write(lines[j].split()[-1] +"\n")

In [ ]:
a=pd.read_csv(out_files+"/hbonds", sep="\t", header=None)

a.columns = ["itype", "chain_a", "chain_b", "resname_a", "resname_b", "resid_a", "resid_b","atom_a", "atom_b" ]
a

In [ ]:
a["resname_a"] = a["resname_a"].astype(str) + a["resid_a"].astype(str)
a["resname_b"] = a["resname_b"].astype(str) + a["resid_b"].astype(str)

b=a.drop(columns=['resid_a','resid_b'])
b

### Intermonomer interactions

In [ ]:
b_inter = b.loc[((b["chain_a"] == "A") & (b["chain_b"] == "B")) | ((b["chain_a"] == "A") & (b["chain_b"] == "C")) ]
b_inter

### Pairwise interactions


In [ ]:
b_inter['pairwise'] = b_inter['resname_a'].str.cat(b_inter['resname_b'],sep="-")
b_inter

### Interaction frequencies

In [ ]:
b_inter['aa_freq'] = b_inter.groupby('resname_a')['resname_a'].transform('count')
b_inter['nuc_freq'] = b_inter.groupby('resname_b')['resname_b'].transform('count')
b_inter['all_freq'] = b_inter.groupby('itype')['itype'].transform('count')
b_inter['pair_freq'] = b_inter.groupby('pairwise')['pairwise'].transform('count')

b_inter

#b_inter_sorted=b_inter.sort_values(('pair_freq') , ascending=False)


## Amino-acid and nucleotide specific interactions

In [ ]:
b_spec_nuc = b_inter.drop(b_inter.index[b_inter['atom_b'].isin(["P","O1P","OP1","OP2","O2P","C5'",
                                                           "O5'","C4'","O4'","C3'","O3'","C2'",
                                                           "C1'","H1'","1H2'","2H2'","H3'","H4'","1H5'","2H5'"])])


b_spec_nuc_and_res = b_spec_nuc.drop(b_spec_nuc.index[b_spec_nuc['atom_a'].isin(["N","CA","C","O","H","HA"])])
b_spec_nuc_and_res

### Recalculate the changed frequency

In [ ]:
b_spec_nuc_and_res['aa_freq'] = b_spec_nuc_and_res.groupby('resname_a')['resname_a'].transform('count')
b_spec_nuc_and_res['nuc_freq'] = b_spec_nuc_and_res.groupby('resname_b')['resname_b'].transform('count')
b_spec_nuc_and_res['all_freq'] = b_spec_nuc_and_res.groupby('itype')['itype'].transform('count')

b_spec_nuc_and_res

## Let's visualize our interaction data

In [ ]:
graphs = os.path.join('graphs')

plt.figure(figsize=(10,8))
ax = sns.barplot(x="pairwise", y="pair_freq", palette="Greens_r", data=b_spec_nuc_and_res)
plt.xticks(rotation='vertical')


plt.xticks(fontsize=13, fontweight='bold')
plt.yticks(fontsize=13, fontweight='bold')
#plt.ylim(ymax = 0.38, ymin = -0.02)
plt.xlabel('Interactions',  fontweight='bold')
plt.ylabel('Number of H-bonds', fontweight='bold')
plt.title("Base-specific H-bond profile", fontweight='bold')

plt.savefig(graphs+"/base-spec-hbond-pairwise.png", dpi=600, bbox_inches='tight', format="png")
